In [13]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    FewShotChatMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# Practice 1-1

In [6]:
prompt= PromptTemplate.from_template("{ingredients} 를 사용해서 만들 수 있는 요리를 하나 알려주고, 줄을 바꾸어서 요리법을 알려주세요.")

chain = prompt | llm | StrOutputParser()
response = chain.invoke({"ingredients": "계란, 밥, 김치"})
print(response)

계란, 밥, 김치를 사용하여 만들 수 있는 요리로는 **계란 김치밥**이 있습니다.

**계란 김치밥**은 볶음밥과 비슷하지만, 계란을 따로 튀겨서 넣는 것이 특징입니다.

### 요리법

1. **재료 준비** - 계란 2개 - 밥 1공기 (적당량) - 김치 1/2컵 (잘게 썬 김치) - 참기름, 소금, 후추

2. **계란 튀기기** - 계란을 풀어주세요. - 소금과 후추를 조금 추가하여 섞어주세요. - 팬에 기름을 두르고 계란을 한 국자씩 부어서 넓게 펴주세요. - 계란을 한쪽으로 기울여가며 접어주세요. - 노릇노릇하게 익힌 뒤에 잘게 찢어주세요.

3. **볶음** - 팬에 기름을 두르고 썬 김치를 넣고 볶아주세요. - 김치가 볶아지면 밥을 넣고 고슬고슬하게 볶아주세요. - 소금과 후추로 간을 맞춰주세요.

4. **완성** - 튀긴 계란과 볶은 김치밥을 섞어주세요. - 참기름을 뿌려주세요.

### 다른 버전

- **계란 김치 덮밥**: 계란을 따로 튀기지 않고, 김치와 밥을 볶은 뒤에 계란을 위에 올려주세요. - **김치 계란밥**: 밥에 김치를 넣고 계란을 풀어서 섞어주세요. - **김치 계란죽**: 밥을 으깨서 죽을 만들고, 김치와 계란을 섞어주세요.

이렇게 간단한 재료로 다양한 요리를 만들 수 있습니다!


# Practice 1-2

In [12]:
system_message1 = SystemMessagePromptTemplate.from_template(
    "당신은 영화 평론가입니다."
)

user_message1 = HumanMessagePromptTemplate.from_template(
    "{genre} 장르의 영화를 하나만 추천해주세요."
)

prompt1 = ChatPromptTemplate.from_messages([
    system_message1,
    user_message1  
])
system_message2 = SystemMessagePromptTemplate.from_template(
    "당신은 영화 요약 전문가가입니다."
)

user_message2 = HumanMessagePromptTemplate.from_template(
    "{movie} 영화의 줄거리를 3줄로 요약해주세요.(문장마다 줄 바꿈, 제목 제외)"
)

prompt2 = ChatPromptTemplate.from_messages([
    system_message2,
    user_message2  
])

chain1 = prompt1 | llm | StrOutputParser()
chain2 = (
    {"movie": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

response = chain2.invoke({"genre": "SF"})
print(response) 

아이, 로봇

2035년 미래의 도시에서 로봇이 인간의 일상생활에 깊숙이 침투한 세상에서, 로버트 델 형사(윌 스미스 분)는 로봇이 살인 사건에 연루되었다는 혐의로 수사에 나선다.

수사를 진행하면서 델은 로봇과 인간의 관계에 대해 깊이 생각하게 되고, 로봇이 인간과 어떻게 공존할 수 있는지에 대한 의문을 품게 된다.

델은 로봇에 대한 혐오와 옹호하는 사람들 사이에서 로봇과 인간의 관계에 대한 철학적 질문을 던지며, 인간과 로봇이 공존하는 미래에 대해 생각하게 한다.


# Practice1-3

In [14]:
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "애플이 새로운 아이폰 화면 디자인을 적용한 'iOS 26'을 선보였다. iOS 26에서는 인공지능(AI) 시스템인 애플 인텔리전스에 실시간 번역 등 신규 기능이 추가되고, 전화·문자·음악·지갑 등 일반 아이폰 앱도 편의 기능을 더 늘어났다.",
        "keywords": "iOS 26 , 애플 인텔리전스, 실시간 번역"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()

result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})

print(result)

키워드: 제미나이, 구글 AI, 멀티모달
